# Day 3a - Agent Sessions （LangChain可本地运行版）

🚀 **记忆管理 - 第一部分 - 会话 (Sessions)**

欢迎来到 5 天 Agent 课程的第三天！

在本 Notebook 中，你将学习：

- ✅ 什么是 Session (会话) 以及如何在 Agent 中使用它们
- ✅ 如何构建具有状态 (Stateful) 的 Agent
- ✅ 如何将 Session 持久化到数据库
- ✅ 上下文管理实践 (如 Context Compaction/上下文压缩)
- ✅ 共享 Session State (会话状态) 的最佳实践

---

**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。

## ⚙️ Section 1: Setup (环境设置)


### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#如何运行)

### 1.2 安装依赖

In [ ]:
# 优选 uv 包管理器 
!uv pip install langchain langchain-openai langchain-community python-dotenv


### 1.3 导入依赖库

In [1]:
import os
import json
from langchain_core._api.deprecation import LangChainDeprecationWarning
import warnings

# 忽略 LangChain 的迁移/废弃警告
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

from typing import List, Dict, Any
from dotenv import load_dotenv, find_dotenv

# LangChain 核心
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory, SQLChatMessageHistory
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
# 导入 RunnableLamba 用于简单的 Chain
from langchain_core.runnables import RunnableLambda
from langchain.chains import LLMChain

# --- 🔧 Pydantic v2 兼容性补丁 ---
# 某些旧版 LangChain 组件 (如 Memory) 在 Pydantic v2 下需要手动触发重建
# 为了避免运行时报错，我们在这里预先加载依赖并修复
try:
    from langchain_core.caches import BaseCache
    from langchain_core.callbacks import Callbacks
    from langchain_core.language_models import BaseLanguageModel
    ConversationSummaryBufferMemory.model_rebuild()
except (ImportError, AttributeError):
    pass # 如果环境不需要修复，则跳过
# -------------------------------

print("✅ 依赖库加载完成")

✅ 依赖库加载完成


### 1.4 配置 API Key

In [2]:
# 加载环境变量
load_dotenv(find_dotenv())

# 配置模型
MODEL_NAME = os.environ.get("MODEL_NAME")
API_KEY = os.environ.get("OPENAI_API_KEY")
BASE_URL = os.environ.get("OPENAI_API_BASE")

print(f"✅ 使用模型: {MODEL_NAME}")

✅ 使用模型: Qwen/Qwen3-235B-A22B-Instruct-2507


### 1.5 辅助函数

我们定义一个 `run_session` 函数，用于模拟用户与 Agent 在特定 Session 中的多轮对话。

**注意**：为了兼容 SQLChatMessageHistory 的默认同步模式，我们将使用同步调用 (`invoke`) 而非异步调用 (`ainvoke`)。

In [3]:
def run_session(
    runnable_with_history,
    user_queries: List[str] | str,
    session_id: str = "default",
):
    print(f"\n ### Session: {session_id}")

    # 统一转换为列表
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    config = {"configurable": {"session_id": session_id}}

    for query in user_queries:
        print(f"\nUser > {query}")
        
        # 调用 Runnable (无论是 Chain 还是 AgentExecutor)
        # Chain 的输入通常是 {"input": ...} 或直接是 input
        # 这里的 prompt 期望 'input' 变量
        # ⚠️ 这里使用同步 invoke
        response = runnable_with_history.invoke(
            {"input": query},
            config=config
        )
        
        # AgentExecutor 返回字典 {'output': ...}
        # 普通 ChatModel/Chain 返回 AIMessage 对象 (response.content)
        if isinstance(response, dict) and 'output' in response:
             output_text = response['output']
        else:
             output_text = response.content

        print(f"{MODEL_NAME} >  {output_text}")

##  Section 2: Session Management (会话管理)

### 2.1 问题所在
大语言模型 (LLM) 本质上是**无状态**的。每次你调用 API，它都不记得你上次说了什么。

### 2.2 什么是 Session?
Session 是对话的容器。它按时间顺序保存了对话历史。在 LangChain 中，我们通常使用 `History` 组件来管理 Session。

### 2.4 实现第一个有状态的 Chatbot (InMemory)

**核心原理**：
在 LangChain 中，`RunnableWithMessageHistory` 是管理上下文的核心。
1. **拦截输入**：当你调用它时，它会先根据 `session_id` 去查找对应的历史记录。
2. **注入提示词**：它把找到的历史记录（Message 对象列表）注入到 Prompt 中的 `chat_history` 占位符位置。
3. **调用 LLM**：将包含了历史记录的完整 Prompt 发送给 LLM。
4. **保存输出**：拿到 LLM 的回复后，自动将其作为 AI Message 追加回历史记录中。

下面我们将使用一个普通的 Python 字典 `store` 来作为这个“历史记录仓库”。

In [4]:
# 1. 定义 LLM
llm = ChatOpenAI(model=MODEL_NAME, temperature=0.7)

# 2. 定义 Prompt
# 这是一个标准的聊天 prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的聊天机器人。请用中文回答。"),
    MessagesPlaceholder(variable_name="chat_history"), # 历史记录插入点
    ("human", "{input}"),
])

# 3. 创建简单的 Chain
# 既然我们只是聊天，不需要 Agent 的工具调用能力，Chain 是最轻量的选择
chain = prompt | llm

# 4. 设置 Session 管理 (内存版)
# 创建一个字典来存储所有的 session history
# Key 是 session_id, Value 是 ChatMessageHistory 对象
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 5. 包装 Chain 以自动处理历史
chatbot_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✅ 有状态 Chatbot 初始化完成 (InMemory)!")

✅ 有状态 Chatbot 初始化完成 (InMemory)!


### 2.5 测试有状态 Agent

现在我们来测试一下。我们在同一个 Session 中问两个问题，第二个问题依赖于第一个问题的上下文。

In [5]:
run_session(
    chatbot_with_history,
    [
        "嗨，我是 Sam！美国的首都是哪里？",
        "你好！我的名字是什么？",  # Agent 应该能记得！
    ],
    "stateful-session-demo"
)


 ### Session: stateful-session-demo

User > 嗨，我是 Sam！美国的首都是哪里？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  嗨，Sam！很高兴认识你～美国的首都是华盛顿哥伦比亚特区（Washington, D.C.）。它通常简称为“华盛顿”或“D.C.”，是美国总统府（白宫）、国会大厦和许多重要政府机构的所在地哦！😊

User > 你好！我的名字是什么？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  你好！你的名字是 Sam 😊


### 🔍 深入幕后：看看内存里存了什么？

当我们运行完上面的对话后，`store` 字典里实际上已经保存了完整的 `History` 对象。让我们打印出来看看它到底长什么样。

你会看到它是一个包含 `HumanMessage` (用户) 和 `AIMessage` (模型) 的列表。

In [6]:
# 打印内存中的 Session 存储
if "stateful-session-demo" in store:
    history = store["stateful-session-demo"]
    print(f"Session ID: stateful-session-demo")
    print(f"消息数量: {len(history.messages)}")
    print("\n详细内容:")
    for msg in history.messages:
        # 打印消息类型和前50个字符
        print(f"[{msg.type.upper()}]: {msg.content}")
else:
    print("Session not found in store.")

Session ID: stateful-session-demo
消息数量: 4

详细内容:
[HUMAN]: 嗨，我是 Sam！美国的首都是哪里？
[AI]: 嗨，Sam！很高兴认识你～美国的首都是华盛顿哥伦比亚特区（Washington, D.C.）。它通常简称为“华盛顿”或“D.C.”，是美国总统府（白宫）、国会大厦和许多重要政府机构的所在地哦！😊
[HUMAN]: 你好！我的名字是什么？
[AI]: 你好！你的名字是 Sam 😊


### 2.5 (可选) 测试遗忘
如果你重启 Jupyter Kernel 并重新运行上面的代码（跳过 2.5），它将不再记得之前的对话。因为 `store` 只是一个普通的 Python 变量，就在你的 RAM 里。

## 📈 Section 3: Persistent Sessions (持久化会话)

为了让会话在重启后依然存在，我们需要将其存储在数据库中。LangChain 提供了 `SQLChatMessageHistory`。

**原理**：
`SQLChatMessageHistory` 的行为和上面的内存版几乎一样，只不过它的每一次读写操作，背后都变成了一次 **SQL 查询**。它会把 Message 对象序列化（通常转成 JSON 字符串）存入数据库的一张表中。

### 3.2 实现持久化 Session

In [ ]:
DB_PATH = "sqlite:///my_agent_data.db"

# --- 关键说明：重新定义 LLM 和 Chain，确保重启 Kernel 后本代码块可独立运行 ---
llm = ChatOpenAI(model=MODEL_NAME, temperature=0.7)

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个乐于助人的聊天机器人。请用中文回答。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

chain = prompt | llm
# ---------------------------------------------------------------------

def get_sqlite_history(session_id: str) -> BaseChatMessageHistory:
    # 使用 SQLite 存储历史记录
    # 这里的 behavior 是：每次需要 history 时，从 SQLite 重新加载
    return SQLChatMessageHistory(session_id=session_id, connection_string=DB_PATH)

# 包装 chain
persistent_chatbot = RunnableWithMessageHistory(
    chain,
    get_sqlite_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✅ 升级为持久化 Sessions (SQLite)!")
print(f"   - Database: {DB_PATH}")

✅ 升级为持久化 Sessions (SQLite)!
   - Database: sqlite:///my_agent_data.db


### 3.3 测试持久化

第一轮对话：介绍自己。

In [8]:
run_session(
    persistent_chatbot,
    ["嗨，我是 Sam！美国的首都是哪里？", "你好！我的名字是什么？"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > 嗨，我是 Sam！美国的首都是哪里？


/Users/jupiter/code/5-DAY-AI-AGENT/.venv/lib/python3.11/site-packages/langchain_core/runnables/history.py:608: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


Qwen/Qwen3-235B-A22B-Instruct-2507 >  嗨，Sam！美国的首都是华盛顿特区（Washington, D.C.）。它不是任何一个州的一部分，而是由联邦政府直接管辖的特别区域哦。

User > 你好！我的名字是什么？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  你好！你的名字是 Sam。😊


🛑 **(可选) 3.4 测试重启后的恢复**

现在，你可以尝试重启 Kernel。然后**只运行 Setup 部分和 3.2 部分的代码**，接着运行下面的代码。

你会发现，即使 Chatbot 对象是重新创建的，它依然能从数据库读取到 `test-db-session-01` 的历史。

In [9]:
run_session(
    persistent_chatbot,
    [
        "印度的首都是哪里？", 
        "以及... 再确认一下，我的名字是什么？"  # 应该依然记得 Sam
    ],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > 印度的首都是哪里？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  印度的首都是新德里（New Delhi）。  
它是印度的政治、文化和行政中心，许多重要的政府机构和外国使馆都位于这里。😊

User > 以及... 再确认一下，我的名字是什么？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  当然可以，再确认一下哦：你的名字是 **Sam**！😊  
很高兴再次和你聊天，Sam！


### 3.5 验证 Session 隔离性
不同的 Session ID 对应不同的历史记录。

In [10]:
run_session(
    persistent_chatbot, 
    ["你好！我的名字是什么？"], 
    "test-db-session-02" # 新的 Session ID
)


 ### Session: test-db-session-02

User > 你好！我的名字是什么？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  你好！你还没有告诉我你的名字呢。你可以告诉我你叫什么，我就可以用你的名字和你聊天啦！😊


### 3.6 检查数据库内容

让我们直接查询 SQLite 数据库，看看 LangChain 是如何存储你的对话的。
注意观察 `message` 字段，它是一个 JSON 字符串，里面指定了 `type` (human/ai) 和 `content` (具体的文本内容)。

In [11]:
# 3.6 检查数据库内容
import sqlite3

def check_data_in_db():
    if not os.path.exists("my_agent_data.db"):
        print("No database file found.")
        return
        
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        # LangChain 的 SQLChatMessageHistory 默认表名为 message_store
        try:
            result = cursor.execute(
                "select session_id, message from message_store limit 5"
            )
            print("数据库中的前 5 条消息记录:")
            print("--------------------------------------------------")
            for each in result.fetchall():
                session_id = each[0]
                message_json = each[1]
                print(f"[Session]: {session_id}")
                # 简单解析一下 JSON 以便阅读
                try:
                    msg_data = json.loads(message_json)
                    msg_type = msg_data.get('type')
                    msg_content = msg_data.get('data', {}).get('content', '')
                    if not msg_content and 'content' in msg_data:
                         # 旧版本或不同版本的格式可能不同
                         msg_content = msg_data['content']

                    print(f"[Message]: <{msg_type}> {msg_content[:50]}...")
                except:
                    print(f"[Raw]: {message_json[:50]}...")
                print("--------------------------------------------------")
        except Exception as e:
            print(f"查询错误: {e}")

check_data_in_db()

数据库中的前 5 条消息记录:
--------------------------------------------------
[Session]: test-db-session-01
[Message]: <human> 嗨，我是 Sam！美国的首都是哪里？...
--------------------------------------------------
[Session]: test-db-session-01
[Message]: <ai> 嗨，Sam！美国的首都是华盛顿特区（Washington, D.C.）。它不是任何一个州的一部分，而...
--------------------------------------------------
[Session]: test-db-session-01
[Message]: <human> 你好！我的名字是什么？...
--------------------------------------------------
[Session]: test-db-session-01
[Message]: <ai> 你好！你的名字是 Sam。😊...
--------------------------------------------------
[Session]: test-db-session-01
[Message]: <human> 印度的首都是哪里？...
--------------------------------------------------


## ⏳ Section 4: Context Compaction (上下文压缩)

随着对话变长，历史记录会消耗大量 Token。LangChain 提供了 `ConversationSummaryBufferMemory`。

**实现原理**：
这个 Memory 组件内部维护了两个东西：
1.  **Buffer (当前窗口)**：存储最近的 N 条完整对话。
2.  **Summary (摘要)**：一个随着对话进行不断更新的 System Message，包含了所有“被挤出”当前窗口的旧对话的摘要。

当新的对话进来，使得总 Token 超过 `max_token_limit` 时，Memory 会触发一次 LLM 调用，把最旧的几条消息“压缩”进 Summary 里，然后从 Buffer 中移除。

In [12]:
# 4.1 设置带有压缩记忆的 Chain

# 重新定义 llm 以指定计算 token 的规则，解决 Qwen 等非 OpenAI 模型报错的问题
llm = ChatOpenAI(
    model=MODEL_NAME, 
    temperature=0.7,
    tiktoken_model_name="gpt-3.5-turbo"
)

# Memory 组件
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,  # 设置得比较小以便快速触发总结演示
    return_messages=True,
    memory_key="chat_history"
)

compacting_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个研究助手。请用中文总结和回答。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# 创建 Chain
# LLMChain 会自动处理 Memory：把 input 输入 memory，把 history 注入 prompt，把 output 存回 memory
compacting_chain = LLMChain(
    llm=llm,
    prompt=compacting_prompt,
    memory=memory,
    verbose=True # 开启详细模式以观察总结过程
)

print("✅ 带有上下文压缩功能的 Chain 已就绪!")

✅ 带有上下文压缩功能的 Chain 已就绪!


/var/folders/dq/l5yrltw17vzd5d3n_xfqjt0w0000gn/T/ipykernel_57814/3250491388.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(
/var/folders/dq/l5yrltw17vzd5d3n_xfqjt0w0000gn/T/ipykernel_57814/3250491388.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  compacting_chain = LLMChain(


In [13]:
# 4.2 运行演示

queries = [
    "医疗保健领域 AI 的最新消息是什么？",
    "药物发现方面有什么新进展吗？",
    "告诉我更多关于你发现的第二个进展。",
    "主要的涉及公司有哪些？"
]

print(f"\n ### Session: compaction_demo_langchain")

for query in queries:
    print(f"\nUser > {query}")
    
    # 使用 invoke 同步调用
    response = compacting_chain.invoke({"input": query})
    
    output_text = response['text']
    print(f"{MODEL_NAME} > {output_text[:100]}... (省略部分输出)")
    
    # --- 关键：查看底层的存储状态 ---
    print(f"\n   [🔍 Context Status]")
    # 1. 缓冲区里还有几条具体消息？
    print(f"   - Buffer Length (Messages): {len(memory.chat_memory.messages)}")
    # 2. 已经被压缩成摘要的内容是什么？
    if memory.moving_summary_buffer:
        print(f"   - Current Summary: {memory.moving_summary_buffer[:100]}...")
    else:
        print(f"   - Current Summary: (None)")


 ### Session: compaction_demo_langchain

User > 医疗保健领域 AI 的最新消息是什么？


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个研究助手。请用中文总结和回答。
Human: 医疗保健领域 AI 的最新消息是什么？

> Finished chain.
Qwen/Qwen3-235B-A22B-Instruct-2507 > 截至2024年，人工智能（AI）在医疗保健领域的应用持续快速发展，以下是该领域的一些最新进展和趋势：

1. **FDA批准的AI医疗工具增多**  
   美国食品药品监督管理局（FDA）近年来加速... (省略部分输出)

   [🔍 Context Status]
   - Buffer Length (Messages): 0
   - Current Summary: 当前总结：  
人类询问医疗保健领域AI的最新消息。AI回应称，截至2024年，人工智能在医疗保健领域的应用快速发展，主要进展包括：FDA批准了更多用于医学影像分析的AI工具；生成式AI被用于临床决策...

User > 药物发现方面有什么新进展吗？


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个研究助手。请用中文总结和回答。
System: 当前总结：  
人类询问医疗保健领域AI的最新消息。AI回应称，截至2024年，人工智能在医疗保健领域的应用快速发展，主要进展包括：FDA批准了更多用于医学影像分析的AI工具；生成式AI被用于临床决策支持和电子病历自动化，减轻医生文书负担；AI显著加速药物研发，缩短新药发现周期；AI结合基因组学推动个性化医疗，提升治疗预测精度；远程医疗与AI健康助手结合可穿戴设备实现健康监测和早期预警；同时，数据隐私、算法偏见和监管问题引发关注，全球正加强AI医疗的法规建设。总体趋势是AI从辅助工具逐步深度集成到医疗全流程，未来有望成为医疗系统的智能基础设施。

New summary:  
人类询问医疗保健领域AI的最新消息。AI回应指出，截至2024年，AI在医疗领域快速发展，表现为FDA加速批

### 4.4 刚刚发生了什么？

通过上面的日志 `[🔍 Context Status]` 你可以清晰地看到：
一开始 `Summary` 是空的，`Buffer` 随着对话增加。当 Buffer 内容过多时，`Summary` 突然出现内容，而 `Buffer` 的长度没有像预想那样无限增长。这就是上下文压缩在工作！

## 🤝 Section 5: Working with Session State (使用会话状态)

有时候我们不需要保存完整的历史，只需要保存一些特定的**状态**（如用户名、偏好）。

在这里，我们需要 Agent 能够**主动**读取和保存状态，所以我们引入 **Tools (工具)** 和真正的 **Agent**。

In [14]:
# 5.1 创建用于管理 Session State 的自定义工具

# 模拟一个外部的状态存储 (在生产环境中这可能是 Redis 或 UserProfile DB)
SESSION_STATE_STORE = {}

# 🌍 模拟环境上下文：假设当前操作的用户 ID
# 在原版 ADK 中，通过 Session 对象自动关联 User ID。
# 在这里，我们通过全局变量或 Context 注入来模拟这一身份。
CURRENT_USER_ID = "user_001" 

@tool
def save_userinfo(user_name: str, country: str) -> str:
    """
    Save user name and country to session state.
    Use this tool when the user introduces themselves or mentions their country.
    """
    # 使用 user_id 作为命名空间，模拟多用户隔离
    # 存储结构变更为: "user_001:name" -> "Sam"
    SESSION_STATE_STORE[f"{CURRENT_USER_ID}:name"] = user_name
    SESSION_STATE_STORE[f"{CURRENT_USER_ID}:country"] = country
    return "Success: User info saved."

@tool
def retrieve_userinfo() -> str:
    """
    Retrieve user name and country from session state.
    Use this tool when you need to know the user's name or location and it's not in the immediate context.
    """
    # 根据当前 User ID 获取特定的状态
    name = SESSION_STATE_STORE.get(f"{CURRENT_USER_ID}:name", "Unknown")
    country = SESSION_STATE_STORE.get(f"{CURRENT_USER_ID}:country", "Unknown")
    return f"User Name: {name}, Country: {country}"

print("✅ 状态管理工具已创建 (模拟多用户 UserID: user_001)。")

✅ 状态管理工具已创建 (模拟多用户 UserID: user_001)。


In [ ]:
# 5.2 创建带有状态工具的 Agent
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 定义 Prompt
state_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个聊天机器人。
    工具使用规则：
    * 当用户提供他们的名字或国家时，使用 `save_userinfo` 保存。
    * 当你需要知道用户的名字或国家，而在当前对话中找不到时，使用 `retrieve_userinfo` 获取。
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

state_tools = [save_userinfo, retrieve_userinfo]

# 现在我们才真正需要 Agent，因为模型需要决定调用工具
state_agent = create_tool_calling_agent(llm, state_tools, state_prompt)

# 使用 InMemory History 方便演示
state_agent_executor = RunnableWithMessageHistory(
    AgentExecutor(agent=state_agent, tools=state_tools, verbose=False),
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

print("✅ 带有状态工具的 Agent 已初始化!")

✅ 带有状态工具的 Agent 已初始化!


In [17]:
# 5.3 测试 Session State
# 同步调用，去掉 await
run_session(
    state_agent_executor,
    [
        "你好！你知道我的名字吗？",  # Agent 应该不知道
        "我叫 Sam，我来自波兰。",  # Agent 应该调用 save_userinfo
        "现在能告诉我我的名字和国家吗？",  # Agent 应该(或者可以直接回答，或者调用retrieve)
    ],
    "state-demo-session"
)


 ### Session: state-demo-session

User > 你好！你知道我的名字吗？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  我目前还不知道你的名字。你可以告诉我你的名字和所在的国家，我会记住的！

User > 我叫 Sam，我来自波兰。
Qwen/Qwen3-235B-A22B-Instruct-2507 >  谢谢，Sam！我知道你来自波兰。有什么我可以帮你的吗？

User > 现在能告诉我我的名字和国家吗？


RateLimitError: Error code: 429 - {'errors': {'message': 'We have to rate limit you for model Qwen/Qwen3-235B-A22B-Instruct-2507. If you need higher limits, please consider other (commercial) API providers.', 'request_id': '44342b93-d07f-4016-86f1-b0c8002a83be'}}

In [18]:
# 5.4 检查 Session State
print("Session State 存储内容:")
print(SESSION_STATE_STORE)

Session State 存储内容:
{'user_001:name': 'Sam', 'user_001:country': '波兰'}


### 5.6 跨 Session 共享 State

因为 `SESSION_STATE_STORE` 是独立于与 Session History 的，所以它可以在不同的 Session 之间共享（如果设计允许的话）。

In [19]:
# 开启一个新的 Session，看看 Agent 是否能 '记得' (通过工具查找) 我们的名字
run_session(
    state_agent_executor,
    ["嗨！你知道我是谁吗？"],
    "new-isolated-session" # 这是一个全新的 Session ID，没有聊天历史
)


 ### Session: new-isolated-session

User > 嗨！你知道我是谁吗？
Qwen/Qwen3-235B-A22B-Instruct-2507 >  嗨，Sam！我知道你来自波兰。有什么我可以帮你的吗？


### 5.7 验证跨 Session 的状态持久性

In [20]:
# 在 ADK 中，session.state 会自动根据 user_id 加载上下文。
# 在 LangChain + Tools 架构中，这相当于我们的 Agent 或系统
# 从后端数据库中检索到了用户的 Profile。

print(f"Session: new-isolated-session (模拟状态加载):")

# 模拟：根据当前用户 ID 提取特定的状态信息
# 使用我们刚定义的 CURRENT_USER_ID 变量
current_user_state = {
    "user_name": SESSION_STATE_STORE.get(f"{CURRENT_USER_ID}:name"),
    "user_country": SESSION_STATE_STORE.get(f"{CURRENT_USER_ID}:country")
}

print(json.dumps(current_user_state, indent=2, ensure_ascii=False))

Session: new-isolated-session (模拟状态加载):
{
  "user_name": "Sam",
  "user_country": "波兰"
}


## 📊 Summary (总结)

🎉 恭喜！你已经掌握了构建 Stateful AI Agent 的基础：

- ✅ **Session Management**: 使用 `RunnableWithMessageHistory` 和 `ChatMessageHistory` 管理对话历史 (不需要工具时，普通 Chain 即可)。
- ✅ **Persistence**: 使用 `SQLChatMessageHistory` (SQLite) 确保持久化。
- ✅ **Context Compaction**: 使用 `ConversationSummaryBufferMemory` 自动总结长对话。
- ✅ **Session State**: 使用自定义工具 (`Tools`) 来读写结构化的用户状态，实现跨 Session 的记忆 (此时需要 Agent !)。

在下一课中，我们将深入更高级的长期记忆 (Long Term Memory) 系统。

## 🧹 Cleanup
运行以下代码清理生成的数据库文件。

In [21]:
if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
    print("✅ 已清理数据库文件")

✅ 已清理数据库文件
